In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-29"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16755,2024-03-29,Japão B2.League,07:00,Ehime Orange Vikings,Fukuoka,2.18,1.64,154.5,1.86,1.80,0.0,0.00,0.00,4Y6q5RsU,0.458716,0.609756,0.537634,0.555556,0.068472,236.744,63.920704,0.269999,1.8,1.643168,0.912871,349.60,2.962,0.804717,0.271680,17.0,100.330,29.889715,0.297914,1.2,1.643168,1.369306,53.76,1.296,0.144845,0.111763,17.0,80,42,4.37,1.28,134.708,126.346,0.199915,0.023184,NaN,1.29,0.258,4.573643,0.494672,0.5,0.005328,1.45,0.290,2.206897,0.577870,0.6,0.022130
16756,2024-03-29,Lituânia Nkl,14:00,Suduva-Mantinga,Kretinga,1.09,6.41,157.5,1.85,1.85,0.0,0.00,0.00,K68F3DGr,0.917431,0.156006,0.540541,0.540541,0.073437,89.086,9.599056,0.107750,1.8,1.643168,0.912871,102.60,1.248,0.093915,0.075252,0.0,507.104,275.449853,0.543182,2.4,1.341641,0.559017,488.37,5.898,2.511926,0.425894,0.0,76,73,1.35,6.69,99.722,475.860,1.003149,0.000000,NaN,-1.00,-0.200,-0.450000,0.833973,0.7,-0.133973,22.97,4.594,1.177623,0.302434,0.6,0.297566
16757,2024-03-29,Lituânia Moteru Lyga Feminina,12:30,Aistes LSU Kaunas F,Siauliai F,7.70,1.07,140.5,1.85,1.85,0.0,0.00,0.00,IJJG2wsd,0.129870,0.934579,0.540541,0.540541,0.064450,576.134,388.918245,0.675048,0.6,1.341641,2.236068,232.44,11.108,9.150949,0.823816,-127.0,268.804,166.420690,0.619115,0.6,1.341641,2.236068,103.84,4.066,2.627666,0.646253,-34.0,52,88,4.47,1.18,513.758,182.740,1.069126,0.000000,NaN,-5.00,-1.000,-6.700000,0.000000,0.0,0.000000,-2.25,-0.450,-0.155556,0.000000,0.0,0.000000
16758,2024-03-29,Portugal Taça Da Liga,17:00,FC Porto,Vitória SC,1.06,7.90,162.5,1.84,1.86,0.0,0.00,0.00,jLkNoYuf,0.943396,0.126582,0.543478,0.537634,0.069979,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,71,72,2.34,1.73,0.000,0.000,1.079601,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
16759,2024-03-29,Polônia Liga De Basquete,13:30,Torun,Dabrowa Gornicza,2.41,1.54,178.5,1.86,1.88,0.0,0.00,0.00,OjVKgOlf,0.414938,0.649351,0.537634,0.531915,0.064288,277.554,138.973430,0.500708,1.2,1.643168,1.369306,167.91,3.432,1.839353,0.535942,-20.0,186.244,82.657568,0.443813,0.6,1.341641,2.236068,311.32,2.202,0.879301,0.399319,-14.0,87,86,1.93,3.62,173.252,208.204,0.311485,0.007563,NaN,-1.70,-0.340,-4.147059,0.512424,0.5,-0.012424,-0.99,-0.198,-2.727273,0.478922,0.6,0.121078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16848,2024-03-29,Chile Lnb,19:00,Espanol Osorno,CD Puerto Varas,1.07,7.50,154.5,1.84,1.84,0.0,0.00,0.00,UV2Js7sp,0.934579,0.133333,0.543478,0.543478,0.067913,122.430,57.089460,0.466303,1.8,1.643168,0.912871,88.14,1.754,1.259377,0.718003,35.0,227.714,191.874082,0.842610,1.2,1.643168,1.369306,81.03,2.948,2.530103,0.858244,1.0,78,73,1.13,1.11,127.976,320.916,1.061073,0.000000,NaN,0.59,0.118,0.593220,0.000000,0.0,0.000000,-1.25,-0.250,-26.000000,0.000000,0.0,0.000000
16849,2024-03-29,Bulgária Nbl,14:00,Minyor,Levski,1.77,1.95,152.5,1.81,1.89,-2.5,1.98,1.92,UNdFLMOE,0.564972,0.512821,0.552486,0.529101,0.077792,193.812,51.685444,0.266678,0.6,1.341641,2.236068,180.60,2.6

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
16759,13:30,Polônia Liga De Basquete,Torun,Dabrowa Gornicza,2.41,Back Home
16772,14:00,Europa Wbbl Feminina,LCC F,Klaipedos Neptunas F,1.57,Back Home
16797,14:30,Romênia Divisão A,Arges Pitesti,Voluntari,1.80,Back Home
16814,22:30,Eua Nba,Utah Jazz,Houston Rockets,2.16,Back Home
16823,20:00,Eua Nba,Indiana Pacers,Los Angeles Lakers,1.74,Back Home
